In [13]:
import os
import numpy as np
import pandas as pd
from math import ceil
from collections import deque
import math
import sys
global maxdays
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\larsv\OneDrive\Documenten\python\COMB


In [14]:
%cd \Users\larsv\OneDrive\Documenten\python\COMB

C:\Users\larsv\OneDrive\Documenten\python\COMB


In [15]:
# Example filepath and filename
directory = "C:\\Users\\larsv\\OneDrive\\Documenten\\python\\COMB"
filename = 'CO_Case2401.txt'


In [16]:

data = pd.read_csv(filename, sep='\t+', header=None, engine='python')
data = data[0].str.split(expand=True)
title = data.iloc[1,2] + ' ' + data.iloc[1,3]
maxdays = data.iloc[2,2]
maxdays = int(maxdays)
constraints = data[[0, 2]].iloc[2:5]
constraints = constraints.set_index(0)[2].to_dict()
constraints = dict((k, int(v)) for k, v in constraints.items())
costs = data[[0, 2]].iloc[5:11]
costs = costs.set_index(0)[2].to_dict()
costs = dict((k, int(v)) for k, v in costs.items())
numberMachines = int(data.iloc[11][2])
index = (11 + numberMachines + 1)  # extra step that ensures the right values are taken
machines = np.array(data[[0, 1, 2]][12:index].astype(int))
numberLocations = int(data.iloc[index][2])
locations = np.array(data[[0, 1, 2]][index + 1:(index + numberLocations + 1)].astype(int))
index += numberLocations + 1  # extra step that ensures the right values are taken
depot = locations[0]
numberRequests = int(data.iloc[index][2])
requests = np.array(data[[0, 1, 2, 3, 4, 5]][index + 1:(index + numberRequests + 1)].astype(int))
index += numberRequests + 1  # extra step that ensures the right values are taken
numberTechnicians = int(data.iloc[index][2])
technicians = np.array(
        data[[i for i in range(0, numberMachines + 4)]][index + 1:(index + numberTechnicians + 1)].astype(int))
constraints

DAYS = constraints['DAYS']
TRUCK_CAPACITY = constraints['TRUCK_CAPACITY']
TRUCK_MAX_DISTANCE = constraints['TRUCK_MAX_DISTANCE']

TRUCK_DISTANCE_COST = costs['TRUCK_DISTANCE_COST']
TRUCK_DAY_COST = costs['TRUCK_DAY_COST']
TRUCK_COST = costs['TRUCK_COST']
TECHNICIAN_DISTANCE_COST = costs['TECHNICIAN_DISTANCE_COST']
TECHNICIAN_DAY_COST = costs['TECHNICIAN_DAY_COST']
TECHNICIAN_COST = costs['TECHNICIAN_COST']
IDLE_MACHINE_COSTS = {machine[0]: machine[2] for machine in machines}

print("Machines:")
print(machines)

print("\nLocations:")
print(locations)

print("\nRequests:")
print(requests)

print("\nTechnicians:")
print(technicians)


print(DAYS)
print(TRUCK_CAPACITY)
print(TRUCK_MAX_DISTANCE)

print(TRUCK_DISTANCE_COST)
print(TRUCK_DAY_COST)
print(TRUCK_COST)
print(TECHNICIAN_DISTANCE_COST)
print(TECHNICIAN_DAY_COST)
print(TECHNICIAN_COST)

Machines:
[[  1   7 315]
 [  2   6 578]]

Locations:
[[  1 230 367]
 [  2 349 370]
 [  3 297 246]
 [  4 495 150]
 [  5 376 373]]

Requests:
[[ 1  3  1  3  1  1]
 [ 2  2  1  4  2  2]
 [ 3  2  1  4  1  2]
 [ 4  4  1  3  1  2]
 [ 5  3  1  2  2  1]
 [ 6  2  1  3  2  2]
 [ 7  3  1  2  1  1]
 [ 8  3  1  3  1  2]
 [ 9  4  1  2  1  1]
 [10  2  1  3  1  2]]

Technicians:
[[  1   3 930   5   1   1]
 [  2   5 898   7   1   1]]
5
15
750
10
250
100000
5
400
20000


In [17]:
def get_location_for_request(request):
    return requests[request-1][1]

def nearest_neighbor(point, unvisited_requests, locations):

    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_request in unvisited_requests:
        request_location_id = get_location_for_request(candidate_request)
        if request_location_id != point:
            candidate_coordinates = get_location_coordinates(request_location_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = request_location_id
    return nearest_point


def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids
for location in locations:
    location_id = location[0]
    location_request  = [get_requests_for_location(requests, location_id)]
    print(location_request)
    

def initialize_home_locations(technicians):
    global home_location_technician
    home_location_technician = {tech[0]: tech[1] for tech in technicians}
    
    # Print the home locations
    print("Home Locations for Technicians:")
    for tech_id, home_location in home_location_technician.items():
        print(f"Technician {tech_id}: Location {home_location}")
    
    return home_location_technician
initialize_home_locations(technicians)
print(home_location_technician)


def nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations):
    home_location_technician = initialize_home_locations(technicians)
    
    for technician_id, home_location in home_location_technician.items():
        nearest_location = nearest_neighbor(home_location, unvisited_requests, locations)
        print(f"The closest location for technician {technician_id} is location {nearest_location}.")
        
        
def transform_skill_set(technicians, technician_id):
    for technician in technicians:
        if technician[0] == technician_id:
            skill_set = technician[4:]  # Get the skill set of the technician (from index 4 onwards)
            transformed_skills = []
            
            # Iterate through the skill_set with index
            for index, value in enumerate(skill_set):
                # If the value is non-zero, append the 1-based index
                if value != 0:
                    transformed_skills.append(index + 1)
                else:
                    # If the value is zero, append zero
                    transformed_skills.append(0)
            
            return transformed_skills
    return None 

def doable_request_skillbased(technicians, requests):
    doable_requests = {}  # Dictionary to store which technician can handle which requests

    for technician in technicians:
        technician_id = technician[0]
        skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
        
        for request in requests:
            request_id = request[0]
            machine_type = request[4]  # Fifth entry denotes the machine type

            # Check if the technician has this machine type in their skill set
            if machine_type in skill_set:
                if technician_id not in doable_requests:
                    doable_requests[technician_id] = []
                doable_requests[technician_id].append(request_id)
    
    return doable_requests


doable_requests = doable_request_skillbased(technicians, requests)
for technician_id, request_ids in doable_requests.items():
    print(f"Technician {technician_id} can handle requests: {request_ids}")
transformed_skills = transform_skill_set(technicians, technician_id)
if transformed_skills is not None:
    print(f"Technician {technician_id}: {transformed_skills}")
else:
    print(f"Technician {technician_id} not found.")

    
def check_location_requests_doable(technicians, requests, locations):
    # Dictionary to store the result for each location
    location_results = {}
    
    for location in locations:
        location_id = location[0]
        location_requests = [request for request in requests if request[1] == location_id]
        location_doable_requests = {}
        
        for technician in technicians:
            technician_id = technician[0]
            skill_set = transform_skill_set(technicians, technician_id)  # Get the transformed skill set of the technician
            
            doable_requests = []
            for request in location_requests:
                machine_type = request[4]  # Fifth entry denotes the machine type
                
                # Check if the technician has this machine type in their skill set
                if machine_type in skill_set:
                    doable_requests.append(request[0])
            
            if doable_requests:
                location_doable_requests[technician_id] = doable_requests
        
        location_results[location_id] = location_doable_requests
    
    return location_results

location_results = check_location_requests_doable(technicians, requests, locations)
for location_id, doable_requests in location_results.items():
    print(f"Location {location_id}:")
    for technician_id, request_ids in doable_requests.items():
        print(f"- Technician {technician_id} can handle requests: {request_ids}")

unvisited_requests = [1, 2, 3, 4,5]
current_location = 2

nearest = nearest_neighbor(current_location, unvisited_requests, locations)
print(f"The closest location for {current_location} is location {nearest}.")

# Call the new method
nearest_neighbor_for_technician_home(technicians, unvisited_requests, locations)


# 2 dingen onthouden, request en locatie id 
# zet locatie, en kijk of de technician deze kan doen


[[]]
[[2, 3, 6, 10]]
[[1, 5, 7, 8]]
[[4, 9]]
[[]]
Home Locations for Technicians:
Technician 1: Location 3
Technician 2: Location 5
{1: 3, 2: 5}
Technician 1 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 2 can handle requests: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Technician 2: [1, 2]
Location 1:
Location 2:
- Technician 1 can handle requests: [2, 3, 6, 10]
- Technician 2 can handle requests: [2, 3, 6, 10]
Location 3:
- Technician 1 can handle requests: [1, 5, 7, 8]
- Technician 2 can handle requests: [1, 5, 7, 8]
Location 4:
- Technician 1 can handle requests: [4, 9]
- Technician 2 can handle requests: [4, 9]
Location 5:
The closest location for 2 is location 3.
Home Locations for Technicians:
Technician 1: Location 3
Technician 2: Location 5
The closest location for technician 1 is location 2.
The closest location for technician 2 is location 2.


In [22]:
def calculate_distance(coordinates1, coordinates2):
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_technician_distances(technician_routes_locations, locations, max_distances, home_location_technician):
    technician_distances = {}
    total_distance_all_technicians = 0  # Initialize total distance for all technicians
    
    for technician_id, route in technician_routes_locations.items():
        total_distance = 0
        prev_location = None
        max_distance = max_distances.get(technician_id, float('inf'))  # Retrieve maximum distance for the current technician
        home_location = home_location_technician[technician_id]  # Get the home location of the technician
        
        for location_id in route:
            if prev_location is not None:
                loc1_id = prev_location - 1  # Subtract 1 to adjust for 0-based indexing
                loc2_id = location_id - 1  # Subtract 1 to adjust for 0-based indexing
                loc1 = np.array(locations[loc1_id][1:])  # Convert to numpy array
                loc2 = np.array(locations[loc2_id][1:])  # Convert to numpy array
                if isinstance(loc1, np.ndarray) and isinstance(loc2, np.ndarray):  # Check if loc1 and loc2 are numpy arrays
                    distance = calculate_distance(loc1, loc2)
                    total_distance += distance
                    if total_distance > max_distance:
                        total_distance -= distance
                        print(f"Technician {technician_id} has exceeded the maximum distance allowed.")
                        break  # Exit the loop if maximum distance is exceeded
            prev_location = location_id
        
        # Add distance from the last location to home
        if route:
            last_location_id = route[-1] - 1
            last_location = np.array(locations[last_location_id][1:])
            home_loc = np.array(locations[home_location - 1][1:])
            distance_to_home = calculate_distance(last_location, home_loc)
            total_distance += distance_to_home
        
        technician_distances[technician_id] = total_distance
        total_distance_all_technicians += total_distance  # Add distance for this technician to total distance
    
    print("Technician distances calculated successfully.")
    print("Total distance for all technicians:", total_distance_all_technicians)
    return technician_distances


In [35]:
def assign_requests(technicians, unvisited_requests, requests, locations):
    # Dictionary to store assigned requests for each technician
    assigned_requests = {}

    # Initialize home locations for technicians
    home_location_technician = initialize_home_locations(technicians)

    # Initialize a global list to track all unvisited requests
    global_unvisited = unvisited_requests[:]

    # Initialize technician details
    print_technician_details(technicians)
    max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
    max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}

    # Helper function to calculate total distance of the route
    def calculate_total_distance(route, locations):
        total_distance = 0
        for i in range(len(route) - 1):
            total_distance += calculate_distance(get_location_coordinates(route[i], locations),
                                                 get_location_coordinates(route[i + 1], locations))
        return total_distance

    # Iterate over each technician
    for technician_id, home_location in home_location_technician.items():
        # Initialize technician's route with home location
        route = [home_location]
        technician_assigned_requests = []

        # Copy global unvisited requests to track visited requests for each technician
        unvisited = global_unvisited[:]

        # Track visited locations to prevent re-visiting the same location
        visited_locations = set()

        # Handle requests at the home location first
        home_requests = get_requests_for_location(requests, home_location)
        for request_id in home_requests:
            if request_id in unvisited:
                unvisited.remove(request_id)
                global_unvisited.remove(request_id)
                technician_assigned_requests.append(request_id)
        
        # Iterate over locations until all requests are visited or constraints are met
        while unvisited and len(technician_assigned_requests) < max_installations[technician_id]:
            # Find the next nearest unvisited location
            current_location = route[-1]
            nearest_request_id = nearest_neighbor(current_location, unvisited, locations)
            if nearest_request_id is None:
                break
            nearest_location = get_location_for_request(nearest_request_id)
            print(f"Technician {technician_id}, Current Location: {current_location}, Nearest Location: {nearest_location}, Unvisited Requests: {unvisited}")

            # Check if adding this location exceeds the maximum distance
            temp_route = route + [nearest_location]
            temp_distance = calculate_total_distance(temp_route, locations)
            if temp_distance > max_distance[technician_id]:
                print(f"Technician {technician_id} cannot add location {nearest_location} due to distance constraints.")
                break

            # Add nearest location to route
            route.append(nearest_location)
            visited_locations.add(nearest_location)
            print(f"Technician {technician_id} route updated: {route}, Distance: {temp_distance}")

            # Remove requests at the visited location from unvisited list
            requests_at_location = get_requests_for_location(requests, nearest_location)
            for request_id in requests_at_location:
                if request_id in unvisited:
                    unvisited.remove(request_id)
                    global_unvisited.remove(request_id)
                    technician_assigned_requests.append(request_id)
                    
                    # Check if the technician has reached the maximum installations
                    if len(technician_assigned_requests) >= max_installations[technician_id]:
                        print(f"Technician {technician_id} has reached the maximum number of installations.")
                        break
            else:
                # Continue if the inner loop wasn't broken
                continue
            # Inner loop was broken, break the outer loop
            break

        # Assign requests for the technician's route
        assigned_requests[technician_id] = route

        # Print the assigned requests for the technician
        print(f"Technician {technician_id} assigned requests: {technician_assigned_requests}")

    # Calculate and print technician distances
    technician_distances = calculate_technician_distances(assigned_requests, locations, max_distance, home_location_technician)
    for tech_id, distance in technician_distances.items():
        print(f"Technician {tech_id} total distance traveled: {distance}")

    return assigned_requests

# Example usage:
assigned_requests = assign_requests(technicians, [1, 2, 3, 4, 5,6,7,8,9,10], requests, locations)
for technician_id, route in assigned_requests.items():
    print(f"Technician {technician_id} route: {route}")


Home Locations for Technicians:
Technician 1: Location 3
Technician 2: Location 5
Technician 1, Current Location: 3, Nearest Location: 2, Unvisited Requests: [2, 3, 4, 6, 9, 10]
Technician 1 route updated: [3, 2], Distance: 134.46189051177288
Technician 1 has reached the maximum number of installations.
Technician 1 assigned requests: [1, 5, 7, 8, 2]
Technician 2, Current Location: 5, Nearest Location: 2, Unvisited Requests: [3, 4, 6, 9, 10]
Technician 2 route updated: [5, 2], Distance: 27.16615541441225
Technician 2, Current Location: 2, Nearest Location: 4, Unvisited Requests: [4, 9]
Technician 2 route updated: [5, 2, 4], Distance: 291.20403148525173
Technician 2 assigned requests: [3, 6, 10, 4, 9]
Technician distances calculated successfully.
Total distance for all technicians: 812.892525235871
Technician 1 total distance traveled: 268.92378102354576
Technician 2 total distance traveled: 543.9687442123252
Technician 1 route: [3, 2]
Technician 2 route: [5, 2, 4]


In [10]:
technician_details = {} 

# Define function to print technician details
def print_technician_details(technicians):
    global technician_details
    technician_details.clear()  # Clear existing data
    for technician in technicians:
        tech_id = technician[0]
        max_distance = technician[2]
        max_installations = technician[3]
        technician_details[tech_id] = {'max_distance': max_distance, 'max_installations': max_installations}


# Call function to print technician details
print_technician_details(technicians)

# Now you can print technician_details
print(technician_details)

# Initialize max_distance and max_installations dictionaries
max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}

{1: {'max_distance': 930, 'max_installations': 5}, 2: {'max_distance': 898, 'max_installations': 7}}


In [11]:
def get_location_for_request(request):
    return requests[request-1][1]

def nearest_neighbor(point, unvisited_requests, locations):

    """
    Find the nearest neighbor of a given point among a list of unvisited points.

    Args:
    - point: The point for which the nearest neighbor is to be found.
    - unvisited_points: List of unvisited points.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - nearest_point: The nearest neighbor of the given point.
    """
    min_distance = float('inf')
    nearest_point = None
    point_coordinates = get_location_coordinates(point, locations)
    
    for candidate_request in unvisited_requests:
        request_location_id = get_location_for_request(candidate_request) # ipv alleen deze id aanmaken, id voor loc en request inhouden
        if request_location_id != int(point):
            candidate_coordinates = get_location_coordinates(request_location_id, locations)
            dist = calculate_distance(point_coordinates, candidate_coordinates)
            if dist is not None and dist < min_distance:
                min_distance = dist
                nearest_point = request_location_id
    return candidate_request
# 2 dingen onthouden, request en locatie id 
# zet locatie, en kijk of de technician deze kan doen

def get_location_coordinates(location_id, locations):
    """
    Get the coordinates of a location based on its ID.

    Args:
    - location_id: The ID of the location.
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - coordinates: Tuple (x, y) representing the coordinates of the location.
    """
    for location in locations:
        if location[0] == location_id:
            return (location[1], location[2])
    return None



def calculate_distance(coordinates1, coordinates2):
    """
    Calculate the Euclidean distance between two points given their coordinates.

    Args:
    - coordinates1: Tuple (x1, y1) representing the coordinates of the first point.
    - coordinates2: Tuple (x2, y2) representing the coordinates of the second point.

    Returns:
    - distance: The Euclidean distance between the two points.
    """
    x1, y1 = coordinates1
    x2, y2 = coordinates2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def calculate_distance_between_requests(request_id1, request_id2, requests, locations):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]

    Returns:
    - distance: The Euclidean distance between the locations of the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None



def get_location_details(requests, locations, request_id):
    """
    Get the location details (ID and coordinates) corresponding to the given request ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id: The request ID for which the location details are to be retrieved.

    Returns:
    - location_details: A tuple containing the location ID and its corresponding coordinates [(location_id, x_coordinate, y_coordinate)]
    - None if the request ID is not found.
    """
    for request in requests:
        if request[0] == request_id:  # Check if request ID matches
            location_id = request[1]
            for location in locations:
                if location[0] == location_id:  # Find location details by matching location ID
                    return (location_id, location[1], location[2])  # Return location details
    return None  # Return None if request ID not found

def calculate_request_distance(requests, locations, request_id1, request_id2):
    """
    Calculate the distance between two requests based on their corresponding location coordinates.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - locations: List of locations where each location is represented as [location_id, x_coordinate, y_coordinate]
    - request_id1: The request ID of the first request.
    - request_id2: The request ID of the second request.

    Returns:
    - distance: The Euclidean distance between the two requests.
    - None if either request ID is not found.
    """
    location1 = get_location_details(requests, locations, request_id1)
    location2 = get_location_details(requests, locations, request_id2)
    
    if location1 and location2:
        coordinates1 = (location1[1], location1[2])
        coordinates2 = (location2[1], location2[2])
        print(f"Location corresponding to Request ID {request_id1}: ID={location1[0]}, Coordinates=({location1[1]}, {location1[2]})")
        print(f"Location corresponding to Request ID {request_id2}: ID={location2[0]}, Coordinates=({location2[1]}, {location2[2]})")
        distance = calculate_distance(coordinates1, coordinates2)
        return distance
    else:
        return None

# Example usage:
request_id1_input = 3
request_id2_input = 5

distance = calculate_request_distance(requests, locations, request_id1_input, request_id2_input)
if distance is not None:
    print(f"Distance between Request ID {request_id1_input} and Request ID {request_id2_input}: {distance:.2f}")
else:
    print("One or both request IDs not found.")
    
    
    
def get_requests_for_location(requests, location_id):
    """
    Get the list of request IDs corresponding to the given location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]
    - location_id: The location ID for which the request IDs are to be retrieved.

    Returns:
    - request_ids: A list of request IDs corresponding to the given location ID.
    """
    request_ids = [req[0] for req in requests if req[1] == location_id]
    return request_ids

unique_location_ids = set(req[1] for req in requests)

# Iterate over unique location IDs
for location_id in unique_location_ids:
    # Get request IDs for the current location ID
    request_ids_for_location = get_requests_for_location(requests, location_id)
    # Print location ID and corresponding request IDs
    print(f"Requests belonging to Location ID {location_id}: {request_ids_for_location}")
    
    
def group_requests_by_location(requests):
    """
    Group requests by their location ID.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, ...]

    Returns:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    location_groups = {}
    for request in requests:
        request_id, location_id = request[0], request[1]
        if location_id not in location_groups:
            location_groups[location_id] = []
        location_groups[location_id].append(request_id)
    return location_groups

def print_location_groups(location_groups):
    """
    Print the location groups.

    Args:
    - location_groups: Dictionary where keys are location IDs and values are lists of request IDs.
    """
    for location_id, request_ids in location_groups.items():
        print(f"Location ID {location_id}: {request_ids}")

location_groups = group_requests_by_location(requests)
print_location_groups(location_groups)



def calculate_request_weight(request_id, requests, machines):
    request = [req for req in requests if req[0] == request_id][0]
    machine_kind_id = request[4]
    num_requested_machines = request[5]
    machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
    return machine_size * num_requested_machines



def check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines):
    depot_loc_ID = depot[0]
    truck_loads = []
    current_truck_load = []
    current_truck_weight = 0
    
    for request_id, location_id, _, _, machine_kind_id, num_requested_machines in requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if current_truck_weight + request_weight > TRUCK_CAPACITY:
            if current_truck_load:
                truck_loads.append(current_truck_load)
            current_truck_load = [(request_id, request_weight)]
            current_truck_weight = request_weight
        else:
            current_truck_load.append((request_id, request_weight))
            current_truck_weight += request_weight
    

# Example usage
check_truck_capacity(locations, depot, TRUCK_CAPACITY, requests, machines)


def get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append([request_id])
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            combined_request_ids = current_truck_route[-1] + [request_id]
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_route[-1] = combined_request_ids
                current_truck_weight = combined_weight
            else:
                truck_routes.append(current_truck_route)
                current_truck_route = [[request_id]]
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # start a new truck route with the current request
            truck_routes.append(current_truck_route)
            current_truck_route = [[request_id]]
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        truck_routes.append(current_truck_route)
    
    return truck_routes


def calculate_delivery_days(truck_routes, requests):
    delivery_days = {}
    for truck_route in truck_routes:
        for location_id, request_ids in enumerate(truck_route, start=1):
            for request_id in request_ids:
                delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
                delivery_days[request_id] = delivery_day  # Record the delivery day for each request
    return delivery_days



# Calculate truck routes
truck_routes = get_truck_routes_with_days(locations, depot, TRUCK_CAPACITY, requests, machines)

# Calculate delivery days using truck routes
delivery_days = calculate_delivery_days(truck_routes, requests)




## weet niet of deze nog klopt na aanpassing delivery/installation days
def calculate_idle_days(requests, delivery_days):
    """
    Calculate the idle days for each machine based on delivery and installation days.

    Args:
    - requests: List of requests where each request is represented as a list [request_id, location_id, delivery_start_day, delivery_end_day, machine_kind_id, num_requested_machines]
    - installations: List of installations where each installation is represented as a list [installation_day, machine_kind_id]

    Returns:
    - idle_days: Dictionary where keys are machine kind IDs and values are lists of idle days.
    """
    idle_days = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]  # Delivery day is at index 2 of each request
        machine_kind_id = requests[request_id - 1][4]  # Machine kind ID is at index 4
        idle_days[machine_kind_id] = max(0, installation_day - delivery_day)
    return idle_days

# Calculate idle days
idle_days = calculate_idle_days(requests, delivery_days)






# Function to calculate penalties for idle days
def calculate_penalties(delivery_days, requests):
    penalties = {}
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        machine_kind_id = requests[request_id - 1][4]
        idle_days = max(0, installation_day - delivery_day - 1)  # Subtract 1 for the day of delivery
        if machine_kind_id not in penalties:
            penalties[machine_kind_id] = 0
        penalties[machine_kind_id] += idle_days
    return penalties


# Function to implement constraints regarding maximum consecutive working days and total distance per day
def apply_constraints(technician_assignments):
    # Implement constraints here
    pass

def get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, requests, machines):
    # Sort requests based on the earliest delivery day
    sorted_requests = sorted(requests, key=lambda req: req[2])
    
    depot_loc_ID = depot[0]
    truck_routes = []
    current_truck_route = []
    current_truck_weight = 0
    
    for request_id, location_id, start_day, end_day, machine_kind_id, num_requested_machines in sorted_requests:
        machine_size = [machine[1] for machine in machines if machine[0] == machine_kind_id][0]
        request_weight = machine_size * num_requested_machines
        
        if not current_truck_route:
            current_truck_route.append(depot_loc_ID)  # Start at the depot
            current_truck_route.append(location_id)  # Add the request location
            current_truck_weight = request_weight
        elif current_truck_weight + request_weight <= TRUCK_CAPACITY:
            # Check if adding the current request exceeds the truck capacity
            # If not, add it to the current truck route
            current_truck_route.append(location_id)  # Add the request location
            combined_weight = current_truck_weight + request_weight
            if combined_weight <= TRUCK_CAPACITY:
                current_truck_weight = combined_weight
            else:
                current_truck_route.append(depot_loc_ID)  # End at the depot
                truck_routes.append(current_truck_route)
                current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
                current_truck_weight = request_weight
        else:
            # If adding the current request exceeds the truck capacity,
            # start a new truck route with the current request
            current_truck_route.append(depot_loc_ID)  # End at the depot
            truck_routes.append(current_truck_route)
            current_truck_route = [depot_loc_ID, location_id]  # Start a new truck route
            current_truck_weight = request_weight
    
    # Append the last truck route if it's not empty
    if current_truck_route:
        current_truck_route.append(depot_loc_ID)  # End at the depot
        truck_routes.append(current_truck_route)
    
    return truck_routes

# Calculate truck routes with depot
truck_routes_with_depot = get_truck_routes_with_depot(locations, depot, TRUCK_CAPACITY, requests, machines)

# Print the truck routes with depot
print("Truck Routes with Depot:")
for idx, truck_route in enumerate(truck_routes_with_depot, start=1):
    print(f"Truck {idx} route: {truck_route}")




def print_truck_routes_with_depot(truck_routes, requests, machines, idle_days, delivery_days, locations):
    total_distance_traveled = 0
    
    for idx, truck_route in enumerate(truck_routes, start=1):
        total_distance_truck = 0
        print(f"Truck {idx} route:")
        total_weight = 0
        
        # Start at depot
        print(f"  Location ID 1 ->", end="")
        
        for request_ids in truck_route:
            for i, request_id in enumerate(request_ids):
                # Print the location ID and request ID
                location_id = requests[request_id - 1][1]
                print(f" Location ID {location_id} ->", end="")
                
                if i < len(request_ids) - 1:
                    print(f" Location ID 1 ->", end="")
                
                if i == len(request_ids) - 1:
                    print(f" Location ID 1")
                    print(f"  Request {request_id}")
                else:
                    print(f" Request {request_id}")
            

# Example usage
print_truck_routes_with_depot(truck_routes, requests, machines, idle_days, delivery_days, locations)

total_distance_traveled = 0

def calculate_truck_distance(locations, route):
    total_distance = 0
    for i in range(len(route) - 1):
        loc1 = locations[route[i] - 1]
        loc2 = locations[route[i + 1] - 1]
        distance = ((loc1[1] - loc2[1]) ** 2 + (loc1[2] - loc2[2]) ** 2) ** 0.5  # Euclidean distance
        total_distance += distance
    return total_distance

def print_truck_routes_with_distances(truck_routes, locations):
    global total_distance_traveled  # Declare as global to modify the variable outside the function
    print("Truck Routes with Distances:")
    for idx, truck_route in enumerate(truck_routes, start=1):
        distance = calculate_truck_distance(locations, truck_route)
        total_distance_traveled += distance
        print(f"Truck {idx} route: {truck_route}, Distance: {distance:.2f}")
    print(f"Total distance traveled by all trucks: {total_distance_traveled:.2f}")

# Example usage
print_truck_routes_with_distances(truck_routes_with_depot, locations)
print(f"Total distance traveled: {total_distance_traveled:.2f}")


# Function to initialize home locations for technicians
def initialize_home_locations(technicians):
    global home_location_technician
    home_location_technician = {tech[0]: tech[1] for tech in technicians}
    
    # Print the home locations
    print("Home Locations for Technicians:")
    for tech_id, home_location in home_location_technician.items():
        print(f"Technician {tech_id}: Location {home_location}")

# Call the function to initialize home locations
initialize_home_locations(technicians)


        
technician_details = {}

# Define function to print technician details
def print_technician_details(technicians):
    global technician_details
    technician_details.clear()  # Clear existing data
    for technician in technicians:
        tech_id = technician[0]
        max_distance = technician[2]
        max_installations = technician[3]
        technician_details[tech_id] = {'max_distance': max_distance, 'max_installations': max_installations}


# Call function to print technician details
print_technician_details(technicians)

# Now you can print technician_details
print(technician_details)

# Initialize max_distance and max_installations dictionaries
max_distance = {tech_id: details['max_distance'] for tech_id, details in technician_details.items()}
max_installations = {tech_id: details['max_installations'] for tech_id, details in technician_details.items()}




def assign_requests_to_technicians(technicians, requests, locations, max_distance, home_location_technician):
    technician_assignments = {tech[0]: [] for tech in technicians}
    technician_routes = {tech[0]: [] for tech in technicians}
    
    unvisited_requests = set(request[0] for request in requests)
    
    while unvisited_requests:
        for technician in technicians:
            technician_id = technician[0]
            home_location_id = home_location_technician[technician_id]
            current_location = home_location_id
            current_day_distance = 0
            day_requests = []

            while unvisited_requests:
                nearest_request = nearest_neighbor(current_location, unvisited_requests, locations)
                print(nearest_request)
                if nearest_request is None:
                    break
                
                request_location_id = requests[nearest_request - 1][1]
                request_coordinates = get_location_coordinates(request_location_id, locations)
                home_coordinates = get_location_coordinates(home_location_id, locations)
                
                distance_to_request = calculate_distance(get_location_coordinates(current_location, locations), request_coordinates)
                distance_back_home = calculate_distance(request_coordinates, home_coordinates)
                
                if current_day_distance + distance_to_request + distance_back_home <= max_distance[technician_id]:
                    day_requests.append(nearest_request)
                    current_day_distance += distance_to_request
                    current_location = request_location_id
                    unvisited_requests.remove(nearest_request)
                else:
                    break

            if day_requests:
                technician_assignments[technician_id].append(day_requests)
                technician_routes[technician_id].append(day_requests)
    
    return technician_assignments, technician_routes

technician_assignments, technician_routes = assign_requests_to_technicians(technicians, requests, locations, max_distance, home_location_technician)



# Print technician assignments
for technician_id, days in technician_assignments.items():
    print(f"Technician {technician_id} Schedule:")
    for day, requests in enumerate(days, start=1):
        print(f"  Day {day}: Requests {requests}")

# Print technician routes
for technician_id, routes in technician_routes.items():
    print(f"Technician {technician_id} Routes:")
    for day, route in enumerate(routes, start=1):
        print(f"  Day {day}: Route {route}")


        

# Define global variable
technician_routes = {}

def generate_technician_routes_of_requests(technician_assignments):
    global technician_routes  # Declare global variable inside the function
    for technician_id, assigned_requests in technician_assignments.items():
        route = [request_id for request_id, _ in assigned_requests]
        technician_routes[technician_id] = route

# Call the function to populate the global variable
generate_technician_routes_of_requests(technician_assignments)

# Print technician routes
for technician_id, route in technician_routes.items():
    print(f"Technician {technician_id} route in requests:", route)
    

# Define global variables
technician_routes_locations = {}

def convert_routes_to_locations(technician_routes, requests):
    global technician_routes_locations  # Declare global variable inside the function
    for technician_id, route in technician_routes.items():
        route_locations = []
        for request_id in route:
            location_id = get_location_details(requests, locations, request_id)[0]
            route_locations.append(location_id)
        technician_routes_locations[technician_id] = route_locations

# Call the function to populate the global variable
convert_routes_to_locations(technician_routes, requests)

def print_technician_routes(technician_routes_locations):
    for technician_id, route in technician_routes_locations.items():
        print(f"Technician {technician_id} route in locations: {route}")

# Print technician routes
print_technician_routes(technician_routes_locations)



def calculate_distance_for_tech(loc1, loc2):
    distance = np.linalg.norm(loc2 - loc1)  # Calculate Euclidean distance between coordinates
    return distance

global total_distance_all_technicians

def calculate_technician_distances(technician_routes_locations, locations):
    global total_distance_all_technicians  # Declare the variable as global
    total_distance_all_technicians = 0  # Initialize total distance for all technicians
    technician_distances = {}
    for technician_id, route in technician_routes_locations.items():
        total_distance = 0
        for i in range(len(route) - 1):
            loc1_id = route[i] - 1  # Subtract 1 to adjust for 0-based indexing
            loc2_id = route[i + 1] - 1  # Subtract 1 to adjust for 0-based indexing
            loc1 = locations[loc1_id][1:]  # Extract coordinates from location
            loc2 = locations[loc2_id][1:]  # Extract coordinates from location
            print(f"Loc1: {loc1}, Loc2: {loc2}")
            if isinstance(loc1, np.ndarray) and isinstance(loc2, np.ndarray):  # Check if loc1 and loc2 are numpy arrays
                distance = calculate_distance_for_tech(loc1, loc2)
                total_distance += distance
        technician_distances[technician_id] = total_distance
        total_distance_all_technicians += total_distance  # Add distance for this technician to total distance
    print(f"Total distance for all technicians: {total_distance_all_technicians}")  # Print total distance
    return technician_distances


# Calculate technician distances
technician_distances = calculate_technician_distances(technician_routes_locations, locations)

# Print technician distances
for technician_id, distance in technician_distances.items():
    print(f"Technician {technician_id} distance: {distance}")

    



penalties = calculate_penalties(delivery_days, requests)
apply_constraints(technician_assignments)

Location corresponding to Request ID 3: ID=2, Coordinates=(349, 370)
Location corresponding to Request ID 5: ID=3, Coordinates=(297, 246)
Distance between Request ID 3 and Request ID 5: 134.46
Requests belonging to Location ID 2: [2, 3, 6, 10]
Requests belonging to Location ID 3: [1, 5, 7, 8]
Requests belonging to Location ID 4: [4, 9]
Location ID 3: [1, 5, 7, 8]
Location ID 2: [2, 3, 6, 10]
Location ID 4: [4, 9]
Truck Routes with Depot:
Truck 1 route: [1, 3, 1]
Truck 2 route: [1, 2, 1]
Truck 3 route: [1, 2, 1]
Truck 4 route: [1, 4, 1]
Truck 5 route: [1, 3, 1]
Truck 6 route: [1, 2, 1]
Truck 7 route: [1, 3, 1]
Truck 8 route: [1, 3, 1]
Truck 9 route: [1, 4, 1]
Truck 10 route: [1, 2, 1]
Truck 1 route:
  Location ID 1 -> Location ID 3 -> Location ID 1
  Request 1
Truck 2 route:
  Location ID 1 -> Location ID 2 -> Location ID 1
  Request 2
Truck 3 route:
  Location ID 1 -> Location ID 2 -> Location ID 1
  Request 3
Truck 4 route:
  Location ID 1 -> Location ID 4 -> Location ID 1
  Request 4

ValueError: too many values to unpack (expected 2)

In [ ]:
def print_schedule_to_file(truck_routes, technician_assignments, delivery_days, requests, idle_days, machines, output_file):
    with open(output_file, 'w') as f:
        f.write("DATASET = ORTEC Caroline VeRoLog 2019\n")
        f.write("NAME = Instance 1\n")
        f.write("\n")
        
        current_day = 1
        unique_technicians_used = set()
        unique_trucks_used = set()
        unique_days_with_technicians = set()
        unique_days_with_trucks = set()
        
        # Calculate total unique technicians and trucks used
        for tech_id, _ in technician_assignments.items():
            unique_technicians_used.add(tech_id)
        
        for route in truck_routes:
            for request_id in route[0]:
                unique_trucks_used.add(request_id)
        
        # Calculate unique days with technicians and trucks
        for _, assignments in technician_assignments.items():
            for _, day in assignments:
                unique_days_with_technicians.add(day)
        
        for route in truck_routes:
            unique_days_with_trucks.add(delivery_days[route[0][0]])
        
        # Print the total counts of unique technicians and trucks used and number of days
        num_unique_technicians_used = len(unique_technicians_used)
        num_unique_trucks_used = len(unique_trucks_used)
        num_days_with_technicians = len(unique_days_with_technicians)
        num_days_with_trucks = len(unique_days_with_trucks)
        f.write(f"TRUCK_DISTANCE: {int(total_distance_traveled)}\n")
        f.write(f"NUMBER_OF_TRUCK_DAYS = {num_days_with_trucks}\n")
        f.write(f"NUMBER_OF_TRUCKS_USED = {num_unique_trucks_used}\n")
        f.write(f"NUMBER_OF_TECHNICIAN_DAYS = {num_days_with_technicians}\n")
        f.write(f"NUMBER_OF_TECHNICIANS_USED = {num_unique_technicians_used}\n\n")
        
        
        # Print the costs for idle machines
        total_idle_cost = 0
        for machine_kind_id, idle_days_count in idle_days.items():
            machine_cost_per_day = [machine[2] for machine in machines if machine[0] == machine_kind_id][0]
            idle_cost = idle_days_count * machine_cost_per_day
            total_idle_cost += idle_cost
        
        f.write(f"IDLE_MACHINE_COSTS = {total_idle_cost}\n\n")
        
        # Print the schedule for each day
        while True:
            f.write("\n")  # Write an empty line before each day
            f.write(f"DAY = {current_day}\n")
            
            # Print truck routes for the current day
            truck_routes_for_day = [route for route in truck_routes if delivery_days[route[0][0]] == current_day]
            num_trucks_for_day = len(truck_routes_for_day)
            if num_trucks_for_day > 0:
                f.write(f"NUMBER_OF_TRUCKS = {num_trucks_for_day}\n")
                for truck_index, route in enumerate(truck_routes_for_day, start=1):
                    request_ids = " ".join(str(request_id) for request_id in route)
                    f.write(f"{truck_index} {request_ids}\n")
            else:
                f.write("NUMBER_OF_TRUCKS = 0\n")
            
            # Print technician assignments for the current day
            technician_assignments_for_day = [(tech_id, request_id) for tech_id, assignments in technician_assignments.items() for request_id, day in assignments if day == current_day]
            num_technicians_for_day = len(set(tech_id for tech_id, request_id in technician_assignments_for_day))
            if num_technicians_for_day > 0:
                f.write(f"NUMBER_OF_TECHNICIANS = {num_technicians_for_day}\n")
                technician_assignments_dict = {}
                for tech_id, request_id in technician_assignments_for_day:
                    if tech_id not in technician_assignments_dict:
                        technician_assignments_dict[tech_id] = []
                    technician_assignments_dict[tech_id].append(str(request_id))
                for tech_id, assigned_requests in technician_assignments_dict.items():
                    f.write(f"{tech_id} {' '.join(assigned_requests)}\n")
            else:
                f.write("NUMBER_OF_TECHNICIANS = 0\n")
            
            # Check if there are no more routes or assignments for the next day
            if not truck_routes_for_day and not technician_assignments_for_day:
                break
            
            current_day += 1

# Example usage
print_schedule_to_file(truck_routes, technician_assignments, delivery_days, requests, idle_days, machines, 'output.txt')


In [12]:
def assign_technicians(delivery_days, technicians, requests, max_consecutive_days, max_distance):
    technician_assignments = {}
    consecutive_days = {}
    
    for request_id, installation_day in delivery_days.items():
        delivery_day = requests[request_id - 1][2]
        earliest_assignment_day = delivery_day + 1
        
        for technician in technicians:
            technician_id = technician[0]
            location_id = technician[1]
            skill_set = technician[4:]
            
            # Check if the technician has the required skill for the request
            if skill_set[requests[request_id - 1][4] - 1] == 1:
                # Calculate the earliest available day for the technician
                earliest_available_day = max(earliest_assignment_day, installation_day)
                
                # Check if the technician has exceeded the maximum consecutive days
                if technician_id in consecutive_days:
                    if consecutive_days[technician_id] >= max_consecutive_days:
                        continue  # Skip assignment if maximum consecutive days exceeded
                
                # Check if the technician has exceeded the maximum distance for the route
                if technician_id in technician_assignments:
                    current_route_distance = calculate_route_distance(technician_assignments[technician_id], requests)
                    if current_route_distance + calculate_request_distance(request_id, requests) > max_distance:
                        continue  # Skip assignment if maximum distance exceeded
                
                # Assign the request to the technician
                if technician_id not in technician_assignments:
                    technician_assignments[technician_id] = []
                    consecutive_days[technician_id] = 0
                
                # Update consecutive days for the technician
                consecutive_days[technician_id] += 1
                
                technician_assignments[technician_id].append((request_id, earliest_available_day))
                break  # Move to the next request after assignment

    return technician_assignments

def calculate_route_distance(route, requests):
    total_distance = 0
    for request_id, _ in route:
        total_distance += calculate_request_distance(request_id, requests)
    return total_distance

def calculate_request_distance(request_id, requests):
    # Replace this function with your implementation to calculate the distance for a request
    # For example, you can use the coordinates of the locations associated with the request
    return 0  # Placeholder value, replace with actual distance calculation

# Example usage:
max_consecutive_days = 5
max_distance = 100  # Example maximum distance allowed for a route
technician_assignments = assign_technicians(delivery_days, technicians, requests, max_consecutive_days, max_distance)


IndexError: invalid index to scalar variable.